# T1047 - Windows Management Instrumentation
Adversaries may abuse Windows Management Instrumentation (WMI) to execute malicious commands and payloads. WMI is an administration feature that provides a uniform environment to access Windows system components. The WMI service enables both local and remote access, though the latter is facilitated by [Remote Services](https://attack.mitre.org/techniques/T1021) such as [Distributed Component Object Model](https://attack.mitre.org/techniques/T1021/003) (DCOM) and [Windows Remote Management](https://attack.mitre.org/techniques/T1021/006) (WinRM).(Citation: MSDN WMI) Remote WMI over DCOM operates using port 135, whereas WMI over WinRM operates over port 5985 when using HTTP and 5986 for HTTPS.(Citation: MSDN WMI)(Citation: FireEye WMI 2015)

An adversary can use WMI to interact with local and remote systems and use it as a means to execute various behaviors, such as gathering information for Discovery as well as remote Execution of files as part of Lateral Movement. (Citation: FireEye WMI SANS 2015) (Citation: FireEye WMI 2015)

## Atomic Tests

### Atomic Test #1 - WMI Reconnaissance UsersAn adversary might use WMI to list all local User Accounts. 
When the test completes , there should be local user accounts information displayed on the command line.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic useraccount get /ALL /format:csv
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 1

### Atomic Test #2 - WMI Reconnaissance ProcessesAn adversary might use WMI to list Processes running on the compromised host.
When the test completes , there should be running processes listed on the command line.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic process get caption,executablepath,commandline /format:csv
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 2

### Atomic Test #3 - WMI Reconnaissance SoftwareAn adversary might use WMI to list installed Software hotfix and patches.
When the test completes, there should be a list of installed patches and when they were installed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic qfe get description,installedOn /format:csv
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 3

### Atomic Test #4 - WMI Reconnaissance List Remote ServicesAn adversary might use WMI to check if a certain Remote Service is running on a remote device. 
When the test completes, a service information will be displayed on the screen if it exists.
A common feedback message is that "No instance(s) Available" if the service queried is not running.
A common error message is "Node - (provided IP or default)  ERROR Description =The RPC server is unavailable" 
if the provided remote host is unreachable
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /node:"127.0.0.1" service where (caption like "%Spooler%")
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 4

### Atomic Test #5 - WMI Execute Local ProcessThis test uses wmic.exe to execute a process on the local host.
When the test completes , a new process will be started locally .A notepad application will be started when input is left on default.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic process call create notepad.exe
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 5

#### Cleanup: 
```cmd
wmic process where name='notepad.exe' delete >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 5 -Cleanup

### Atomic Test #6 - WMI Execute Remote ProcessThis test uses wmic.exe to execute a process on a remote host. Specify a valid value for remote IP using the node parameter.
To clean up, provide the same node input as the one provided to run the test
A common error message is "Node - (provided IP or default)  ERROR Description =The RPC server is unavailable" if the default or provided IP is unreachable
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /user:DOMAIN\Administrator /password:P@ssw0rd1 /node:"127.0.0.1" process call create notepad.exe
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 6

#### Cleanup: 
```cmd
wmic /user:DOMAIN\Administrator /password:P@ssw0rd1 /node:"127.0.0.1" process where name='notepad.exe' delete >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Create a Process using WMI Query and an Encoded CommandSolarigate persistence is achieved via backdoors deployed via various techniques including using PowerShell with an EncodedCommand
 Powershell -nop -exec bypass -EncodedCommand <encoded command>
Where the –EncodedCommand, once decoded, would resemble:
  Invoke-WMIMethod win32_process -name create -argumentlist ‘rundll32 c:\windows\idmu\common\ypprop.dll _XInitImageFuncPtrs’ -ComputerName WORKSTATION
The EncodedCommand in this atomic is the following: Invoke-WmiMethod -Path win32_process -Name create -ArgumentList notepad.exe
You should expect to see notepad.exe running after execution of this test.
[Solarigate Analysis from Microsoft](https://www.microsoft.com/security/blog/2020/12/18/analyzing-solorigate-the-compromised-dll-file-that-started-a-sophisticated-cyberattack-and-how-microsoft-defender-helps-protect/)
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
powershell -exec bypass -e SQBuAHYAbwBrAGUALQBXAG0AaQBNAGUAdABoAG8AZAAgAC0AUABhAHQAaAAgAHcAaQBuADMAMgBfAHAAcgBvAGMAZQBzAHMAIAAtAE4AYQBtAGUAIABjAHIAZQBhAHQAZQAgAC0AQQByAGcAdQBtAGUAbgB0AEwAaQBzAHQAIABuAG8AdABlAHAAYQBkAC4AZQB4AGUA
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 7

### Atomic Test #8 - Create a Process using obfuscated Win32_ProcessThis test tries to mask process creation by creating a new class that inherits from Win32_Process. Indirect call of suspicious method such as Win32_Process::Create can break detection logic.
[Cybereason blog post No Win32_ProcessNeeded](https://www.cybereason.com/blog/wmi-lateral-movement-win32)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$Class = New-Object Management.ManagementClass(New-Object Management.ManagementPath("Win32_Process"))
$NewClass = $Class.Derive("Win32_Atomic")
$NewClass.Put()
Invoke-WmiMethod -Path Win32_Atomic -Name create -ArgumentList notepad.exe
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 8

#### Cleanup: 
```powershell
$CleanupClass = New-Object Management.ManagementClass(New-Object Management.ManagementPath("Win32_Atomic"))
try { $CleanupClass.Delete() } catch {}
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 8 -Cleanup

### Atomic Test #9 - WMI Execute rundll32
This test uses wmic.exe to execute a DLL function using rundll32. Specify a valid value for remote IP using the node parameter.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL with function to execute must exist on disk at specified location (#{dll_to_execute})
##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\calc.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1047/bin/calc.dll?raw=true" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\calc.dll"

```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /node:127.0.0.1 process call create "rundll32.exe \"PathToAtomicsFolder\..\ExternalPayloads\calc.dll\" StartW"
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 9

#### Cleanup: 
```cmd
taskkill /f /im calculator.exe```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Application uninstall using WMIC
Emulates uninstalling  applications using WMIC.  This method only works if the product was installed with an msi file.  APTs have been seen using this to uninstall security products.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: TightVNC must be installed.
##### Check Prereq Commands:
```cmd
if ((Test-Path "C:\Program Files\TightVNC\tvnviewer.exe")-Or (Test-Path "C:\Program Files (x86)\TightVNC\tvnviewer.exe")) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest 'https://www.tightvnc.com/download/2.8.63/tightvnc-2.8.63-gpl-setup-64bit.msi' -OutFile "PathToAtomicsFolder..\ExternalPayloads\tightvncinstaller.msi"
start-sleep -s 10
msiexec /i "PathToAtomicsFolder..\ExternalPayloads\tightvncinstaller.msi" /qn /norestart
start-sleep -s 15
```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /node:"127.0.0.1" product where "name like 'Tightvnc%%'" call uninstall```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 10

#### Cleanup: 
```cmd
msiexec /i "PathToAtomicsFolder..\ExternalPayloads\tightvncinstaller.msi" /qn /norestart```

In [ ]:
Invoke-AtomicTest T1047 -TestNumbers 10 -Cleanup

## Detection
Monitor network traffic for WMI connections; the use of WMI in environments that do not typically use WMI may be suspect. Perform process monitoring to capture command-line arguments of "wmic" and detect commands that are used to perform remote behavior. (Citation: FireEye WMI 2015)


## Shield Active Defense
### Admin Access 
 Modify a user's administrative privileges.
 

 Changing the target system to allow or disallow users to perform tasks requiring administrator level permissions gives the defender leverage in inhibiting or facilitating attacks.  The procedures for changing these permissions vary across different operating and software systems.
#### Opportunity
In an adversary engagement scenario, there is an opportunity to allow or restrict admin access to support your defensive objectives.
#### Use Case
A defender can remove admin access from the local user to prevent an adversary from being able to utilize WMI.
#### Procedures
Remove an account's administrative access from a system or service to require an adversary to reveal techniques for elevating privileges in order to accomplish certain tasks.
Grant an account administrative access to a system or service to enable an adversary to take advantage of those privileges if they compromise the system or service.
